# Problems Jupyter Notebook

## Problem 1: Extending the Lady Tasting Tea
![The Lady Tasting Tea](https://upload.wikimedia.org/wikipedia/en/2/2d/The_Lady_Tasting_Tea_-_David_Salsburg.jpg)

### Objective: 
_Let's extend the Lady Tasting Tea experiment as follows. The original experiment has 8 cups: 4 tea-first and 4 milk-first. Suppose we prepare 12 cups: 8 tea-first and 4 milk-first. A participant claims they can tell which was poured first._

_Simulate this experiment using `numpy` by randomly shuffling the cups many times and calculating the probability of the participant correctly identifying all cups by chance. Compare your result with the original 8-cup experiment._

_In your notebook, explain your simulation process clearly, report and interpret the estimated probability, and discuss whether, based on this probability, you would consider extending or relaxing the p-value threshold compared to the original design._

In [ ]:
# Import libraries that will be used:
import math
import itertools
import numpy as np
import matplotlib.pyplot as plt

## Problem 2: Normal Distribution

## Problem 3: t-Tests

## Problem 4: ANOVA

### References:
- A very helpful github guide on `markdown` used throughout for all markdown formatting [here](https://github.com/adam-p/markdown-here/wiki/markdown-cheatsheet)

## END